In [854]:
# import our dependencies

%matplotlib inline
from collections import Counter
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import numpy as np
import os

In [855]:
file = 'Beverly_Grove_Cleaned.csv'

location = '/Users/Admin/Desktop/GW_Bootcamp/Analysis_Projects/Final_Project_Team_1/'

In [856]:
path = os.path.join(location, file)

In [857]:
# import our file

Beverly_Grove_df = pd.read_csv(path, index_col='PARCEL')
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,6727 Drexel Ave,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,942 S Cochran Ave,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,0,0


In [858]:
Beverly_Grove_df = Beverly_Grove_df.dropna()

In [859]:
Beverly_Grove_df.dtypes

Owned by Trust?                int64
Owned by Business?             int64
SITEADDRESS                   object
MAIL DIFFERENT FROM SITE?      int64
MAIL OUTSIDE CA?               int64
TITLECO1                       int64
ASSDTOTAL                      int64
ASSDLAND                       int64
ASSDSTCT                       int64
ASSDOTHR                       int64
EXEMPTCD                       int64
EXMPTAMT                       int64
PCNTIMPD                       int64
TAXAMT                       float64
DOCDATE_YEAR                   int64
MULTIPARCEL                    int64
PRICE                          int64
LENDER1                        int64
LOANAMOUT1                     int64
LOANTYPE1                      int64
YEARBLT                        int64
EFFYRBLT                     float64
LOTSQFT                        int64
TOTALSF                        int64
GRGTYPE                        int64
BLDGMAT                        int64
TOTUNITS                       int64
Q

In [860]:
Beverly_Grove_df['TAXAMT'] = Beverly_Grove_df['TAXAMT'].astype(int)
Beverly_Grove_df['EFFYRBLT'] = Beverly_Grove_df['EFFYRBLT'].astype(int)

In [861]:
# Make sure our dataframe is only SFR's so exclude every property that has an '#' in its site address

Beverly_Grove_df = Beverly_Grove_df[~Beverly_Grove_df['SITEADDRESS'].str.contains('#')]
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0


In [862]:
# Now drop the SiteAddress column
Beverly_Grove_df.drop(['SITEADDRESS'], axis=1, inplace=True)

In [863]:
# check to see if it dropped.
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,0,0,0,0,0,0,0,0,0,0


In [864]:
len(Beverly_Grove_df)

4705

# Attempt 1: Run a standard Logistic Regression

In [865]:
# Step 1: Create our X and y

y = Beverly_Grove_df['Did it sell?']
X = Beverly_Grove_df.drop(['Did it sell?'], 1)

In [866]:
APN = Beverly_Grove_df.index.values
APN

array([5085010022, 5085033008, 5085032022, ..., 5510018012, 5510017023,
       5514001029])

In [867]:
# Split our data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

In [868]:
X_train.head(3)

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5088017017,0,0,0,0,1,340000,136000,204000,0,1,...,0,0,0,0,0,0,0,0,0,0
5089020010,0,0,0,0,0,218091,106556,111535,0,1,...,0,0,0,0,0,0,0,0,0,0
5088016028,1,0,0,0,1,195815,49267,146548,0,1,...,0,0,0,0,0,0,0,0,0,0


In [869]:
X_train.columns

Index(['Owned by Trust?', 'Owned by Business?', 'MAIL DIFFERENT FROM SITE?',
       'MAIL OUTSIDE CA?', 'TITLECO1', 'ASSDTOTAL', 'ASSDLAND', 'ASSDSTCT',
       'ASSDOTHR', 'EXEMPTCD', 'EXMPTAMT', 'PCNTIMPD', 'TAXAMT',
       'DOCDATE_YEAR', 'MULTIPARCEL', 'PRICE', 'LENDER1', 'LOANAMOUT1',
       'LOANTYPE1', 'YEARBLT', 'EFFYRBLT', 'LOTSQFT', 'TOTALSF', 'GRGTYPE',
       'BLDGMAT', 'TOTUNITS', 'QUALCLAS', 'BLDGCOND', 'NOSTORY', 'ROOFMAT',
       'FOUNDATN', 'BEDROOMS', 'BATHROOMS', 'FAMILYRM', 'DININGRM', 'POOL',
       'PATIO', 'FIREPLCE', 'AIRMTHOD', 'HEATMTHD', 'VIEW', 'GRANT DEED',
       'IN LIEU OF FORC', 'INTERSPOUSAL', 'JOINT TENANT', 'NAME CHANGE',
       'Other', 'PARTNERSHIP', 'PERSONAL REP', 'QUIT CLAIM', 'RE-RECORD',
       'TAX DEED', 'TRUST TRANSFER', 'TRUSTEES', 'WARRANTY'],
      dtype='object')

In [870]:
# Scale our data

# Create the StandardScaler instance
scaler = StandardScaler()

# Fit our scaler, named 'scaler' to our data, which produces a new StandardScaler object
# which we call 'X_scaler'
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [871]:
X_train_scaled_APN = []

In [872]:
for index, parcel in enumerate(X_train.index.values):
     X_train_scaled_APN.append(np.insert(X_train_scaled[index], 0, [parcel]))

In [873]:
# Create the learning model

classifier = LogisticRegression(solver='lbfgs',
                                max_iter=300,
                                random_state=78,
                                class_weight="balanced")

In [874]:
# fit our learning model to our scaled training data

classifier.fit(X_train_scaled, y_train)

LogisticRegression(class_weight='balanced', max_iter=300, random_state=78)

In [875]:
y_pred = classifier.predict(X_test_scaled)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test, "PARCEL": y_test.index.values}).reset_index(drop=True)
results.sample(20)

,Prediction,Actual,PARCEL
237,0,0,5087009006
169,0,0,5087023024
1067,0,0,5510002022
220,0,0,5510020024
322,0,1,5085020006
125,0,0,5087006012
521,0,0,5512018034
430,1,0,5086004005
812,0,0,5087012013
1094,0,0,5526022019


In [876]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.5921835174171622


In [877]:
from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(y_test, y_pred))

0.4951901414796792


In [878]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average="weighted")

0.6923828010441901

In [879]:
# See our results in a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[666, 430],
       [ 50,  31]])

In [880]:
# evaluate
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.61      0.74      1096
           1       0.07      0.38      0.11        81

    accuracy                           0.59      1177
   macro avg       0.50      0.50      0.42      1177
weighted avg       0.87      0.59      0.69      1177



In [881]:
Beverly_Grove_df = Beverly_Grove_df.merge(results, how="left", right_on="PARCEL", left_on="PARCEL")
Beverly_Grove_df.head()

,PARCEL,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY,Prediction,Actual
0,5085010022,0,0,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,NaN,NaN
1,5085033008,0,0,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0.0,0.0
2,5085032022,0,0,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,5510005006,1,0,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,0.0,0.0
4,5089024002,0,0,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,NaN,NaN


# Attempt 2: Resample our data using SMOTEEN

In [882]:
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({0: 2095, 1: 3245})

In [883]:
model_2 = LogisticRegression(solver='lbfgs', random_state=78, max_iter=300, class_weight="balanced")
model_2.fit(X_resampled, y_resampled)

LogisticRegression(class_weight='balanced', max_iter=300, random_state=78)

In [884]:
# Calculated the balanced accuracy score
y_pred = model_2.predict(X_test_scaled)

In [885]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.5283635216725241

In [886]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.5794392523364486

In [887]:
f1_score(y_test, y_pred, average="weighted")

0.6818244603198075

In [888]:
confusion_matrix(y_test, y_pred)

array([[644, 452],
       [ 43,  38]])

In [889]:
# evaluate
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.59      0.72      1096
           1       0.08      0.47      0.13        81

    accuracy                           0.58      1177
   macro avg       0.51      0.53      0.43      1177
weighted avg       0.88      0.58      0.68      1177



# Attempt 3: Resample our Data using SMOTE

In [890]:
from imblearn.over_sampling import SMOTE
X_resampled_2, y_resampled_2 = SMOTE(random_state=1, 
                               sampling_strategy='auto').fit_resample(
                               X_train_scaled, y_train)

In [891]:
Counter(y_resampled_2)

Counter({0: 3287, 1: 3287})

In [892]:
# perform our logistic regression

model_3 = LogisticRegression(solver='lbfgs', random_state=78, max_iter=300, class_weight="balanced")
model_3.fit(X_resampled_2, y_resampled_2)

LogisticRegression(class_weight='balanced', max_iter=300, random_state=78)

In [893]:
# Calculated the balanced accuracy score
y_pred = model_3.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.5146492295214923

In [894]:
accuracy_score(y_test, y_pred)

0.5751911639762107

In [895]:
f1_score(y_test, y_pred, average="weighted")

0.6785693826192267

In [896]:
confusion_matrix(y_test, y_pred)

array([[641, 455],
       [ 45,  36]])

# Attempt 4: Use a Support Vector Machine

In [897]:
# Instantiate a linear SVM model
from sklearn.svm import SVC
model_4 = SVC(kernel='linear')

In [898]:
#model_4.class_weight_

In [899]:
#model_4.coef_

In [900]:
#pd.Series(abs(model_4.coef_[0]), index=X_train.columns).nlargest(20).plot(kind='barh')

In [901]:
Beverly_Grove_df.columns

Index(['PARCEL', 'Owned by Trust?', 'Owned by Business?',
       'MAIL DIFFERENT FROM SITE?', 'MAIL OUTSIDE CA?', 'TITLECO1',
       'ASSDTOTAL', 'ASSDLAND', 'ASSDSTCT', 'ASSDOTHR', 'EXEMPTCD', 'EXMPTAMT',
       'PCNTIMPD', 'TAXAMT', 'DOCDATE_YEAR', 'MULTIPARCEL', 'PRICE', 'LENDER1',
       'LOANAMOUT1', 'LOANTYPE1', 'YEARBLT', 'EFFYRBLT', 'LOTSQFT', 'TOTALSF',
       'GRGTYPE', 'BLDGMAT', 'TOTUNITS', 'QUALCLAS', 'BLDGCOND', 'NOSTORY',
       'ROOFMAT', 'FOUNDATN', 'BEDROOMS', 'BATHROOMS', 'FAMILYRM', 'DININGRM',
       'POOL', 'PATIO', 'FIREPLCE', 'AIRMTHOD', 'HEATMTHD', 'VIEW',
       'Did it sell?', 'GRANT DEED', 'IN LIEU OF FORC', 'INTERSPOUSAL',
       'JOINT TENANT', 'NAME CHANGE', 'Other', 'PARTNERSHIP', 'PERSONAL REP',
       'QUIT CLAIM', 'RE-RECORD', 'TAX DEED', 'TRUST TRANSFER', 'TRUSTEES',
       'WARRANTY', 'Prediction', 'Actual'],
      dtype='object')

In [902]:
#model_4.support_vectors_

In [903]:
# Fit/train our model using the scaled training data
model_4.fit(X_train_scaled, y_train)

SVC(kernel='linear')

In [904]:
# Make predictions using the test data
y_pred = model_4.predict(X_test_scaled)
results_SVM = pd.DataFrame({
    "Prediction": y_pred,
    "Actual": y_test
}).reset_index(drop=True)

In [905]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9311809685641461

In [906]:
balanced_accuracy_score(y_test, y_pred)

0.5

In [907]:
f1_score(y_test, y_pred, average="weighted")

0.8979976608414467

In [908]:
# create our confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1096,    0],
       [  81,    0]])

In [909]:
# evaluate
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96      1096
           1       0.00      0.00      0.00        81

    accuracy                           0.93      1177
   macro avg       0.47      0.50      0.48      1177
weighted avg       0.87      0.93      0.90      1177



/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

## Attempt 5: Reduce the number of columns to just the important ones and run an SVC

In [910]:
# REimport our file

# Beverly_Grove_df = pd.read_csv('Beverly_Grove_Cleaned.csv', index_col='PARCEL')
# Beverly_Grove_df.head()

In [911]:
Beverly_Grove_Reduced_df = Beverly_Grove_df[['MAIL DIFFERENT FROM SITE?',
                                             'ASSDTOTAL',
                                             'ASSDLAND',
                                             'ASSDSTCT',
                                             'BLDGCOND',
                                             'TAXAMT',
                                             'PRICE',
                                             'YEARBLT',
                                             'EFFYRBLT',
                                             'LOANAMOUT1',
                                             'Did it sell?']]

In [912]:
#Beverly_Grove_Reduced_df.set_index(['PARCEL'], inplace=True)

In [913]:
Beverly_Grove_Reduced_df.head(3)

,MAIL DIFFERENT FROM SITE?,ASSDTOTAL,ASSDLAND,ASSDSTCT,BLDGCOND,TAXAMT,PRICE,YEARBLT,EFFYRBLT,LOANAMOUT1,Did it sell?
0,0,1157385,925908,231477,1,14144,1140000,1905,1938,775200,0
1,1,57047,40994,16053,0,858,0,1906,1916,0,0
2,0,651485,521193,130292,0,8044,0,1909,1937,0,0


In [914]:
len(Beverly_Grove_Reduced_df)

4705

In [915]:
# create new X and y values based on the Beverly_Grove_Reduced dataframe

y_2 = Beverly_Grove_Reduced_df['Did it sell?']
X_2 = Beverly_Grove_Reduced_df[['MAIL DIFFERENT FROM SITE?',
                              'ASSDTOTAL',
                              'ASSDLAND',
                              'ASSDSTCT',
                              'BLDGCOND',
                              'TAXAMT',
                              'PRICE',
                              'YEARBLT',
                              'EFFYRBLT',
                              'LOANAMOUT1']]

In [916]:
# create new training and testing data

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, random_state=78, stratify=y)

In [917]:
# Scale our new training and test data

# Create the StandardScaler instnce
scaler_2 = StandardScaler()

In [918]:
# Fit our scaler, named 'scaler_2' to our data, which produced a new 
# Standard Scaler object, which we call X_scaler_2
X_scaler_2 = scaler_2.fit(X_train_2)

In [919]:
# Scale the data
X_train_2_scaled = X_scaler_2.transform(X_train_2)
X_test_2_scaled = X_scaler_2.transform(X_test_2)

In [920]:
# Instantiate a linear SVM model
from sklearn.svm import SVC
model_5 = SVC(kernel='linear')

In [921]:
# Fit/train our model using the scaled training data
model_5.fit(X_train_2_scaled, y_train_2)

SVC(kernel='linear')

In [922]:
# Make predictions using the test data
y_pred_2 = model_5.predict(X_test_2_scaled)
results_SVM = pd.DataFrame({
    "Prediction": y_pred_2,
    "Actual": y_test_2
}).reset_index(drop=True)

In [923]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test_2, y_pred_2)

0.9311809685641461

In [924]:
balanced_accuracy_score(y_test_2, y_pred_2)

0.5

In [925]:
f1_score(y_test_2, y_pred_2, average="weighted")

0.8979976608414467

In [926]:
cm = confusion_matrix(y_test_2, y_pred_2)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [927]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print("Classification Report")
print(classification_report(y_test_2, y_pred_2))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1096,0
Actual 1,81,0


Classification Report
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      1096
           1       0.00      0.00      0.00        81

    accuracy                           0.93      1177
   macro avg       0.47      0.50      0.48      1177
weighted avg       0.87      0.93      0.90      1177



/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

## Attempt 6: Decision Tree

In [928]:
from sklearn import tree

# Instantiate our learning model
model_6 = tree.DecisionTreeClassifier()

In [929]:
# Fit our model to our trained, scaled data
model_6 = model_6.fit(X_train_scaled, y_train)

In [930]:
# Make predictions using the testing data
predictions = model_6.predict(X_test_scaled)

# check your predictions
predictions[:20]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [931]:
# Evaluate our model
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)
acc_score

0.8632115548003398

In [932]:
# balanced accuracy score
balanced_accuracy_score(y_test, predictions)

0.5149533657745337

In [933]:
# generate the F1 score
f1_score(y_test, predictions, average="weighted")

0.8691725199612143

In [934]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1007,89
Actual 1,72,9


Accuracy Score : 0.8632115548003398
Classification Report
              precision    recall  f1-score   support

           0       0.93      0.92      0.93      1096
           1       0.09      0.11      0.10        81

    accuracy                           0.86      1177
   macro avg       0.51      0.51      0.51      1177
weighted avg       0.88      0.86      0.87      1177



## Attempt 7: Random Forest

In [935]:
from sklearn.ensemble import RandomForestClassifier
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)

# Remember, the accuracy_score function requires 2 arguments, the test and the pred
print(f" Random forest predictive accuracy: {accuracy_score(y_test, y_pred):.3f}")

 Random forest predictive accuracy: 0.931


In [936]:
acc_score = accuracy_score(y_test, y_pred)

In [937]:
balanced_accuracy_score(y_test, y_pred)

0.5

In [938]:
# generate the F1 score
f1_score(y_test, predictions, average="weighted")

0.8691725199612143

In [939]:
cm_RandomForest = confusion_matrix(y_test, y_pred)
cm_RandomForest_df = pd.DataFrame(
    cm_RandomForest, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [940]:
# Displaying results
print("Confusion Matrix")
display(cm_RandomForest_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1096,0
Actual 1,81,0


Accuracy Score : 0.9311809685641461
Classification Report
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      1096
           1       0.00      0.00      0.00        81

    accuracy                           0.93      1177
   macro avg       0.47      0.50      0.48      1177
weighted avg       0.87      0.93      0.90      1177



/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

# Attempt 7b: RandomForest with reduced columns

In [941]:
# Use these columns: 'LOTSQFT', 'TOTALSF', 'PARCEL', 'DOCDATE_YEAR', 'ASSDLAND', 'ASSDSTCT', 'EFFYRBLT', 'TAXAMT', 'ASSDTOTAL', 'YEARBLT', 'PCNTIMPD', 'LOANAMOUT1', 'PRICE', 'BEDROOMS'

In [942]:
file = 'Beverly_Grove_Cleaned.csv'

location = '/Users/Admin/Desktop/GW_Bootcamp/Analysis_Projects/Final_Project_Team_1/'

In [943]:
path = os.path.join(location, file)

In [944]:
# import our file

Beverly_Grove_df = pd.read_csv(path, index_col='PARCEL')
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,6727 Drexel Ave,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,942 S Cochran Ave,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,0,0


In [945]:
Beverly_Grove_df = Beverly_Grove_df.dropna()

In [946]:
Beverly_Grove_df['TAXAMT'] = Beverly_Grove_df['TAXAMT'].astype(int)
Beverly_Grove_df['EFFYRBLT'] = Beverly_Grove_df['EFFYRBLT'].astype(int)

In [947]:
# Make sure our dataframe is only SFR's so exclude every property that has an '#' in its site address

Beverly_Grove_df = Beverly_Grove_df[~Beverly_Grove_df['SITEADDRESS'].str.contains('#')]
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0


In [948]:
# Now drop the SiteAddress column
Beverly_Grove_df.drop(['SITEADDRESS'], axis=1, inplace=True)

In [949]:
Beverly_Grove_Reduced_df = pd.DataFrame(Beverly_Grove_df, columns = ['LOTSQFT', 'TOTALSF', 'PARCEL', 'DOCDATE_YEAR', 'ASSDLAND', 'ASSDSTCT', 'EFFYRBLT', 'TAXAMT', 'ASSDTOTAL', 'YEARBLT', 'PCNTIMPD', 'LOANAMOUT1', 'PRICE', 'BEDROOMS', 'Did it sell?'])

In [950]:
Beverly_Grove_Reduced_df = Beverly_Grove_Reduced_df.drop(['PARCEL'], axis=1)
Beverly_Grove_Reduced_df.head()

,LOTSQFT,TOTALSF,DOCDATE_YEAR,ASSDLAND,ASSDSTCT,EFFYRBLT,TAXAMT,ASSDTOTAL,YEARBLT,PCNTIMPD,LOANAMOUT1,PRICE,BEDROOMS,Did it sell?
PARCEL,,,,,,,,,,,,,,
5085010022,7134,1895,2014,925908,231477,1938,14144,1157385,1905,20,775200,1140000,3,0
5085033008,6034,1108,1980,40994,16053,1916,858,57047,1906,28,0,0,2,0
5085032022,7844,2004,2007,521193,130292,1937,8044,651485,1909,20,0,0,3,0
5510005006,11510,2778,1988,74539,11158,1927,1325,85697,1909,13,0,0,4,0
5089024002,6689,1568,2008,42013,20474,1924,876,62487,1910,33,0,0,3,0


In [951]:
Beverly_Grove_Reduced_df.dtypes

LOTSQFT         int64
TOTALSF         int64
DOCDATE_YEAR    int64
ASSDLAND        int64
ASSDSTCT        int64
EFFYRBLT        int64
TAXAMT          int64
ASSDTOTAL       int64
YEARBLT         int64
PCNTIMPD        int64
LOANAMOUT1      int64
PRICE           int64
BEDROOMS        int64
Did it sell?    int64
dtype: object

In [952]:
# Step 1: Create our X and y

y = Beverly_Grove_Reduced_df['Did it sell?']
X = Beverly_Grove_Reduced_df.drop(['Did it sell?'], 1)

In [953]:
APN = Beverly_Grove_Reduced_df.index.values
APN

array([5085010022, 5085033008, 5085032022, ..., 5510018012, 5510017023,
       5514001029])

In [954]:
# Split our data into training and testing sets

X_train_Reduced, X_test_Reduced, y_train_Reduced, y_test_Reduced = train_test_split(X, y, random_state=78, stratify=y)

In [955]:
X_train_Reduced.head(3)

,LOTSQFT,TOTALSF,DOCDATE_YEAR,ASSDLAND,ASSDSTCT,EFFYRBLT,TAXAMT,ASSDTOTAL,YEARBLT,PCNTIMPD,LOANAMOUT1,PRICE,BEDROOMS
PARCEL,,,,,,,,,,,,,
5088017017,9827,2841,2012,136000,204000,1928,4358,340000,1927,60,990000,0,4
5089020010,9456,2365,1988,106556,111535,1931,2851,218091,1928,51,0,0,3
5088016028,7500,3188,1999,49267,146548,1940,2533,195815,1926,75,0,0,4


In [956]:
# Scale our data

# Create the StandardScaler instance
scaler = StandardScaler()

# Fit our scaler, named 'scaler' to our data, which produces a new StandardScaler object
# which we call 'X_scaler'
X_scaler = scaler.fit(X_train_Reduced)

# Scale the data
X_train_scaled = X_scaler.transform(X_train_Reduced)
X_test_scaled = X_scaler.transform(X_test_Reduced)

In [957]:
from sklearn.ensemble import RandomForestClassifier
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train_Reduced)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)

# Remember, the accuracy_score function requires 2 arguments, the test and the pred
print(f" Random forest predictive accuracy: {accuracy_score(y_test_Reduced, y_pred):.3f}")

 Random forest predictive accuracy: 0.929


In [958]:
balanced_accuracy_score(y_test_Reduced, y_pred)

0.4990875912408759

In [959]:
# generate the F1 score
f1_score(y_test_Reduced, y_pred, average="weighted")

0.8971483748209387

In [960]:
cm_RandomForest = confusion_matrix(y_test_Reduced, y_pred)
cm_RandomForest_df = pd.DataFrame(
    cm_RandomForest, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [961]:
# Displaying results
print("Confusion Matrix")
display(cm_RandomForest_df)
print("Classification Report")
print(classification_report(y_test_Reduced, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1094,2
Actual 1,81,0


Classification Report
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      1096
           1       0.00      0.00      0.00        81

    accuracy                           0.93      1177
   macro avg       0.47      0.50      0.48      1177
weighted avg       0.87      0.93      0.90      1177



# Attempt 8: Gradient Boosting

In [962]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error

In [963]:
# Choose the best learning rate

learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=learning_rate,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)

    # Fit the model
    classifier.fit(X_train_scaled, y_train)
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train)))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test)))
    print()

Learning rate:  0.05
Accuracy score (training): 0.932
Accuracy score (validation): 0.931

Learning rate:  0.1
Accuracy score (training): 0.932
Accuracy score (validation): 0.931

Learning rate:  0.25
Accuracy score (training): 0.939
Accuracy score (validation): 0.927

Learning rate:  0.5
Accuracy score (training): 0.943
Accuracy score (validation): 0.928

Learning rate:  0.75
Accuracy score (training): 0.941
Accuracy score (validation): 0.925

Learning rate:  1
Accuracy score (training): 0.943
Accuracy score (validation): 0.911



In [964]:
# Looks like a learning rate of 0.25 has the best results. 

classifier = GradientBoostingClassifier(n_estimators=20,
                                        learning_rate=0.5,
                                        max_features=5,
                                        max_depth=3,
                                        random_state=0)

# Fit the model
classifier.fit(X_train_scaled, y_train)

# Make Prediction
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).sample(20)

,Prediction,Actual
PARCEL,,
5526015015,0,1
5088007031,0,0
5510017030,0,0
5514010006,0,0
5511005010,0,0
5513020006,0,0
5089012004,0,0
5089025022,0,0
5089026023,0,0


In [965]:
# Evaluate the model

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.9277824978759558


In [966]:
balanced_accuracy_score(y_test, predictions)

0.5038918176083627

In [967]:
# generate the F1 score
f1_score(y_test, predictions, average="weighted")

0.8978488633767664

In [968]:
# Generate the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying results
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,1091,5
Actual 1,80,1


# Attempt 9: Adaptive Boosting

In [969]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [970]:
# lets use the training data from the reduced dataframe (the one with fewer columns)

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2), n_estimators=200,
    algorithm='SAMME.R', learning_rate=0.5)

# fit our model to our scaled data
ada_clf.fit(X_train_2_scaled, y_train_2)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2),
                   learning_rate=0.5, n_estimators=200)

In [971]:
# Evaluate the model
y_pred_3 = ada_clf.predict(X_test_2_scaled)

In [972]:
# Remember, the accuracy_score function requires 2 arguments, the test and the pred
print(f" Adaptive Boosting predictive accuracy: {accuracy_score(y_test_2, y_pred_3):.3f}")

 Adaptive Boosting predictive accuracy: 0.912


In [973]:
balanced_accuracy_score(y_test_2, y_pred_3)

0.5013967739028566

In [974]:
f1_score(y_test_2, y_pred_3, average="weighted")

0.8910693347762086

In [975]:
cm_AdaBoost = confusion_matrix(y_test_2, y_pred_3)
cm_AdaBoost_df = pd.DataFrame(
    cm_AdaBoost, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [976]:
# Displaying results
print("Confusion Matrix")
display(cm_AdaBoost_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test_2, y_pred_3))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1072,24
Actual 1,79,2


Accuracy Score : 0.9277824978759558
Classification Report
              precision    recall  f1-score   support

           0       0.93      0.98      0.95      1096
           1       0.08      0.02      0.04        81

    accuracy                           0.91      1177
   macro avg       0.50      0.50      0.50      1177
weighted avg       0.87      0.91      0.89      1177



# Attempt 10: Use K-Folds & Then Logistic Regression

In [977]:
# partition our data into training and holdout sets
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import max_error

#X_train_KFolds, X_holdout_KFolds, y_train_KFolds, y_holdout_KFolds = train_test_split(X, y, stratify=y, random_state=78, shuffle=True)

In [978]:
# from sklearn.model_selection import KFold

# for train_index, test_index in KFold(n_splits=5).split(X_train_KFolds):
#     X_data = X_train_KFolds.iloc[train_index]
#     y_data = y_train_KFolds.iloc[train_index]

In [979]:
max_err = 0

for train_index, test_index in KFold(n_splits = 5).split(X):

    model = RandomForestClassifier(
        # our hyperparameters, except n_jobs
        # n_jobs = -1,
        n_estimators = 300,
        max_depth = 5,
        max_features = 'sqrt'
    )
    
    model.fit(X.iloc[train_index], y.iloc[train_index])
    y_pred_KFolds = model.predict(X.iloc[test_index])
    max_err += max_error(y_pred_KFolds, y.iloc[test_index])
    
max_err / 5

1.0

In [980]:
model.score(X, y)

0.9315621679064825

In [981]:
len(test_index)

941

In [982]:
accuracy_score(y.iloc[test_index], y_pred_KFolds)

0.9298618490967057

In [983]:
from sklearn.metrics import classification_report
print(classification_report(y.iloc[test_index], y_pred_KFolds))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96       875
           1       0.00      0.00      0.00        66

    accuracy                           0.93       941
   macro avg       0.46      0.50      0.48       941
weighted avg       0.86      0.93      0.90       941



/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Admin/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [984]:
#print(f" KFolds + Random Forest accuracy: {accuracy_score(y_holdout_KFolds, y_pred_KFolds):.3f}")

# Attempt 11: Use a Logistic Regression, but use unscaled data

In [985]:
# Split our data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

In [986]:
# Create the learning model

classifier = LogisticRegression(solver='lbfgs',
                                max_iter=300,
                                random_state=78,
                                class_weight="balanced")

In [987]:
# fit our learning model to our scaled training data

classifier.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=300, random_state=78)

In [988]:
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test, "PARCEL": y_test.index.values}).reset_index(drop=True)
results.sample(20)

,Prediction,Actual,PARCEL
725,0,0,5527032009
354,0,1,5089018008
694,1,0,5085003006
659,1,0,4305003025
613,1,0,5089019008
1034,1,0,5084032020
139,0,0,5085006019
816,0,0,5525035024
299,1,0,4334023012
550,0,0,5514007024


In [989]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.5488530161427357


In [990]:
balanced_accuracy_score(y_test, y_pred)

0.5233734342615122

In [991]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average="weighted")

0.6565100061280048

In [992]:
# See our results in a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[606, 490],
       [ 41,  40]])

In [993]:
# evaluate
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.55      0.70      1096
           1       0.08      0.49      0.13        81

    accuracy                           0.55      1177
   macro avg       0.51      0.52      0.41      1177
weighted avg       0.88      0.55      0.66      1177



# Attempt 12: PCA + Logistic Regression

In [994]:
# bring in our dataframe

Beverly_Grove_df = pd.read_csv(path, index_col='PARCEL')
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,6727 Drexel Ave,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,942 S Cochran Ave,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,0,0


In [995]:
# drop any nulls

Beverly_Grove_df = Beverly_Grove_df.dropna()

In [996]:
Beverly_Grove_df['TAXAMT'] = Beverly_Grove_df['TAXAMT'].astype(int)
Beverly_Grove_df['EFFYRBLT'] = Beverly_Grove_df['EFFYRBLT'].astype(int)

In [997]:
# Make sure our dataframe is only SFR's so exclude every property that has an '#' in its site address

Beverly_Grove_df = Beverly_Grove_df[~Beverly_Grove_df['SITEADDRESS'].str.contains('#')]
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0


In [998]:
# Now drop the SiteAddress column
Beverly_Grove_df.drop(['SITEADDRESS'], axis=1, inplace=True)

In [999]:
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,0,0,0,0,0,0,0,0,0,0


In [1000]:
# Step 1: Create our X and y

y = Beverly_Grove_df['Did it sell?']
X = Beverly_Grove_df.drop(['Did it sell?'], 1)

In [1001]:
APN = Beverly_Grove_df.index.values
APN

array([5085010022, 5085033008, 5085032022, ..., 5510018012, 5510017023,
       5514001029])

In [1002]:
# Step 2: Use PCA to reduce dimension to three principal components.
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

In [1003]:
X_pca

array([[ 1037442.40088322,   222949.06197812,    99008.44211244],
       [ -932464.7141857 ,   251848.62995472,   -90439.60157538],
       [ -393042.03210882,  -290764.40544306,   -27460.47099801],
       ...,
       [ -314557.49024403,  -369458.72763661,   -27020.9155848 ],
       [ 2038277.22062523,   419857.55993186, -1288530.60059949],
       [ -962055.78924987,   281605.81570128,   -93574.2647082 ]])

In [1004]:
# Step 3: Train, test, split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, random_state=78, stratify=y)

In [1005]:
# Step 4: Scale our data

# Create the StandardScaler instance
scaler = StandardScaler()

# Fit our scaler, named 'scaler' to our data, which produces a new StandardScaler object
# which we call 'X_scaler'
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [1006]:
X_train_scaled_APN = []

In [1007]:
# for index, parcel in enumerate(X_train.index.values):
#      X_train_scaled_APN.append(np.insert(X_train_scaled[index], 0, [parcel]))

In [1008]:
# Step 5:Create the learning model

log_classifier = LogisticRegression(solver='lbfgs',
                                max_iter=300,
                                random_state=78,
                                class_weight="balanced")

# Train the model
log_classifier.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test_scaled)

print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.613


In [1009]:
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test, "PARCEL": y_test.index.values}).reset_index(drop=True)
results.sample(20)

,Prediction,Actual,PARCEL
989,0,0,5512008015
314,0,0,5514009001
964,0,0,5085014028
750,0,1,5087019015
937,1,1,5087012028
142,0,0,5089026024
237,0,0,5087009006
225,1,0,5086017028
228,1,0,5511034014
576,0,0,5510020025


In [1010]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[693, 403],
       [ 53,  28]])

In [1011]:
balanced_accuracy_score(y_test, y_pred)

0.48898914120933584

In [1012]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average="weighted")

0.7081876800594455

In [1013]:
# evaluate
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.63      0.75      1096
           1       0.06      0.35      0.11        81

    accuracy                           0.61      1177
   macro avg       0.50      0.49      0.43      1177
weighted avg       0.87      0.61      0.71      1177



# Attempt 13: PCA + Oversampling (SMOTE) + LogisticRegression

In [1014]:
# bring in our dataframe

Beverly_Grove_df = pd.read_csv(path, index_col='PARCEL')
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,6727 Drexel Ave,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,942 S Cochran Ave,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,0,0


In [1015]:
# drop any nulls

Beverly_Grove_df = Beverly_Grove_df.dropna()

In [1016]:
# Change the datatype of these two columns into ints

Beverly_Grove_df['TAXAMT'] = Beverly_Grove_df['TAXAMT'].astype(int)
Beverly_Grove_df['EFFYRBLT'] = Beverly_Grove_df['EFFYRBLT'].astype(int)

In [1017]:
# Make sure our dataframe is only SFR's so exclude every property that has an '#' in its site address

Beverly_Grove_df = Beverly_Grove_df[~Beverly_Grove_df['SITEADDRESS'].str.contains('#')]
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0


In [1018]:
# Now drop the SiteAddress column
Beverly_Grove_df.drop(['SITEADDRESS'], axis=1, inplace=True)

In [1019]:
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,0,0,0,0,0,0,0,0,0,0


In [1020]:
# Step 1: Create our X and y

y = Beverly_Grove_df['Did it sell?']
X = Beverly_Grove_df.drop(['Did it sell?'], 1)

In [1021]:
APN = Beverly_Grove_df.index.values
APN

array([5085010022, 5085033008, 5085032022, ..., 5510018012, 5510017023,
       5514001029])

In [1022]:
# Step 2: Use PCA to reduce dimension to three principal components.
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

In [1023]:
X_pca

array([[ 1037442.40088321,   222949.06197815,    99008.44211241],
       [ -932464.7141857 ,   251848.62995472,   -90439.60157538],
       [ -393042.03210882,  -290764.40544306,   -27460.470998  ],
       ...,
       [ -314557.49024403,  -369458.72763661,   -27020.9155848 ],
       [ 2038277.22062523,   419857.55993186, -1288530.60059949],
       [ -962055.78924987,   281605.81570128,   -93574.2647082 ]])

In [1024]:
# Step 3: Train, test, split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, random_state=78, stratify=y)

In [1025]:
# Step 4: Scale our data

# Create the StandardScaler instance
scaler = StandardScaler()

# Fit our scaler, named 'scaler' to our data, which produces a new StandardScaler object
# which we call 'X_scaler'
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [1026]:
X_train_scaled_APN = []

In [1027]:
# Step 5: Resample our data using the SMOTE method

from imblearn.over_sampling import SMOTE
X_resampled_SMOTE, y_resampled_SMOTE = SMOTE(random_state=1, 
                               sampling_strategy='auto').fit_resample(
                               X_train_scaled, y_train)

In [1028]:
Counter(y_resampled_SMOTE)

Counter({0: 3287, 1: 3287})

In [1029]:
# Step 6: Instantiate our model and fit it to our data

model_7 = LogisticRegression(solver='lbfgs', random_state=78, max_iter=300, class_weight="balanced")
model_7.fit(X_resampled_SMOTE, y_resampled_SMOTE)

LogisticRegression(class_weight='balanced', max_iter=300, random_state=78)

In [1030]:
accuracy_score(y_test, y_pred)

0.6125743415463042

In [1031]:
# Step 7: Calculated the balanced accuracy score

y_pred = model_7.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.4914841849148418

In [1032]:
print(f" PCA + Smote + Logistic Regression Was: {accuracy_score(y_test, y_pred):.3f}")

 PCA + Smote + Logistic Regression Was: 0.628


In [1033]:
f1_score(y_test, y_pred, average="weighted")

0.719691707488016

In [1034]:
confusion_matrix(y_test, y_pred)

array([[712, 384],
       [ 54,  27]])

In [1035]:
# evaluate
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.65      0.76      1096
           1       0.07      0.33      0.11        81

    accuracy                           0.63      1177
   macro avg       0.50      0.49      0.44      1177
weighted avg       0.87      0.63      0.72      1177



# Attempt 14: PCA + Resampling (SMOTEEN) + LogisticRegression

In [1036]:
# bring in our dataframe

Beverly_Grove_df = pd.read_csv(path, index_col='PARCEL')
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,6727 Drexel Ave,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,942 S Cochran Ave,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,0,0


In [1037]:
# drop any nulls

Beverly_Grove_df = Beverly_Grove_df.dropna()

In [1038]:
# Change the datatype of these two columns into ints

Beverly_Grove_df['TAXAMT'] = Beverly_Grove_df['TAXAMT'].astype(int)
Beverly_Grove_df['EFFYRBLT'] = Beverly_Grove_df['EFFYRBLT'].astype(int)

In [1039]:
# Make sure our dataframe is only SFR's so exclude every property that has an '#' in its site address

Beverly_Grove_df = Beverly_Grove_df[~Beverly_Grove_df['SITEADDRESS'].str.contains('#')]
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0


In [1040]:
# Now drop the SiteAddress column
Beverly_Grove_df.drop(['SITEADDRESS'], axis=1, inplace=True)

In [1041]:
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,0,0,0,0,0,0,0,0,0,0


In [1042]:
# Step 1: Create our X and y

y = Beverly_Grove_df['Did it sell?']
X = Beverly_Grove_df.drop(['Did it sell?'], 1)

In [1043]:
APN = Beverly_Grove_df.index.values
APN

array([5085010022, 5085033008, 5085032022, ..., 5510018012, 5510017023,
       5514001029])

In [1044]:
# Step 2: Use PCA to reduce dimension to three principal components.
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

In [1045]:
X_pca

array([[ 1037442.40088322,   222949.0619781 ,    99008.44211245],
       [ -932464.7141857 ,   251848.62995472,   -90439.60157539],
       [ -393042.03210882,  -290764.40544306,   -27460.470998  ],
       ...,
       [ -314557.49024403,  -369458.72763661,   -27020.9155848 ],
       [ 2038277.22062523,   419857.55993186, -1288530.60059949],
       [ -962055.78924987,   281605.81570128,   -93574.2647082 ]])

In [1046]:
# Step 3: Train, test, split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, random_state=78, stratify=y)

In [1047]:
# Step 4: Scale our data

# Create the StandardScaler instance
scaler = StandardScaler()

# Fit our scaler, named 'scaler' to our data, which produces a new StandardScaler object
# which we call 'X_scaler'
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [1048]:
X_train_scaled_APN = []

In [1049]:
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({0: 1542, 1: 1893})

In [1050]:
model_8 = LogisticRegression(solver='lbfgs', random_state=78, max_iter=300, class_weight="balanced")
model_8.fit(X_resampled, y_resampled)

LogisticRegression(class_weight='balanced', max_iter=300, random_state=78)

In [1051]:
# Calculated the balanced accuracy score
y_pred = model_8.predict(X_test_scaled)

In [1052]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.4869221411192214

In [1053]:
print(f" PCA + SMOTEEN + Logistic Regression Accuracy was: {accuracy_score(y_test, y_pred):.3f}")

 PCA + SMOTEEN + Logistic Regression Accuracy was: 0.619


In [1054]:
f1_score(y_test, y_pred, average="weighted")

0.7133305329304763

In [1055]:
confusion_matrix(y_test, y_pred)

array([[702, 394],
       [ 54,  27]])

In [183]:
# evaluate
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.64      0.76      1096
           1       0.06      0.33      0.11        81

    accuracy                           0.62      1177
   macro avg       0.50      0.49      0.43      1177
weighted avg       0.87      0.62      0.71      1177



# Attempt 15: PCA + Undersampling (Centroid Clustering) + Logistic Regression

In [184]:
# bring in our dataframe

Beverly_Grove_df = pd.read_csv(path, index_col='PARCEL')
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,6727 Drexel Ave,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,942 S Cochran Ave,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,0,0


In [185]:
# drop any nulls

Beverly_Grove_df = Beverly_Grove_df.dropna()

In [186]:
# Change the datatype of these two columns into ints

Beverly_Grove_df['TAXAMT'] = Beverly_Grove_df['TAXAMT'].astype(int)
Beverly_Grove_df['EFFYRBLT'] = Beverly_Grove_df['EFFYRBLT'].astype(int)

In [187]:
# Make sure our dataframe is only SFR's so exclude every property that has an '#' in its site address

Beverly_Grove_df = Beverly_Grove_df[~Beverly_Grove_df['SITEADDRESS'].str.contains('#')]
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0


In [188]:
# Now drop the SiteAddress column
Beverly_Grove_df.drop(['SITEADDRESS'], axis=1, inplace=True)

In [189]:
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,0,0,0,0,0,0,0,0,0,0


In [190]:
# Step 1: Create our X and y

y = Beverly_Grove_df['Did it sell?']
X = Beverly_Grove_df.drop(['Did it sell?'], 1)

In [191]:
APN = Beverly_Grove_df.index.values
APN

array([5085010022, 5085033008, 5085032022, ..., 5510018012, 5510017023,
       5514001029])

In [192]:
# Step 2: Use PCA to reduce dimension to three principal components.
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

In [193]:
X_pca

array([[ 1037442.40088322,   222949.06197809,    99008.44211246],
       [ -932464.7141857 ,   251848.62995471,   -90439.60157539],
       [ -393042.03210882,  -290764.40544306,   -27460.470998  ],
       ...,
       [ -314557.49024403,  -369458.72763661,   -27020.9155848 ],
       [ 2038277.22062523,   419857.55993186, -1288530.60059949],
       [ -962055.78924987,   281605.81570128,   -93574.2647082 ]])

In [194]:
# Step 3: Train, test, split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, random_state=78, stratify=y)

In [195]:
# Step 4: Scale our data

# Create the StandardScaler instance
scaler = StandardScaler()

# Fit our scaler, named 'scaler' to our data, which produces a new StandardScaler object
# which we call 'X_scaler'
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [196]:
X_train_scaled_APN = []

In [197]:
# Fit the data using `ClusterCentroids` and check the count of each class
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_CC, y_CC = cc.fit_resample(X_train_scaled, y_train)
Counter(y_CC)

Counter({0: 241, 1: 241})

In [198]:
# Logistic regression using cluster centroid undersampled data
from sklearn.linear_model import LogisticRegression
model_9 = LogisticRegression(solver='lbfgs', random_state=78)
model_9.fit(X_CC, y_CC)

LogisticRegression(random_state=78)

In [199]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model_9.predict(X_test_scaled)
confusion_matrix(y_test, y_pred)

array([[229, 867],
       [ 18,  63]])

In [200]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.49335969180859696

In [201]:
print(f" PCA + Centroid Clustering + Logistic Regression was: {accuracy_score(y_test, y_pred):.3f}")

 PCA + Centroid Clustering + Logistic Regression was: 0.248


In [202]:
f1_score(y_test, y_pred, average="weighted")

0.3261352171412433

In [203]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.93      0.21      0.78      0.34      0.40      0.15      1096
          1       0.07      0.78      0.21      0.12      0.40      0.17        81

avg / total       0.87      0.25      0.74      0.33      0.40      0.15      1177



In [204]:
# Note how using the Centroid Clusters creates a model that predicts a much higher number of sales. 
# When we decrease the number of non-sale properties, the model curiously predicts more sale properties. 
# What does this imply? Is it that the sale properties are indistinct from the non-sales, and evening out the classes, 
# creates an increased likelihood predicting sale? 

## Attempt 16: PCA + Oversampling (Random Oversampling) + Logistic Regression

In [554]:
# bring in our dataframe

Beverly_Grove_df = pd.read_csv(path, index_col='PARCEL')
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,6727 Drexel Ave,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,942 S Cochran Ave,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,0,0


In [555]:
# drop any nulls

Beverly_Grove_df = Beverly_Grove_df.dropna()

In [556]:
# Change the datatype of these two columns into ints

Beverly_Grove_df['TAXAMT'] = Beverly_Grove_df['TAXAMT'].astype(int)
Beverly_Grove_df['EFFYRBLT'] = Beverly_Grove_df['EFFYRBLT'].astype(int)

In [557]:
# Make sure our dataframe is only SFR's so exclude every property that has an '#' in its site address

Beverly_Grove_df = Beverly_Grove_df[~Beverly_Grove_df['SITEADDRESS'].str.contains('#')]
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0


In [558]:
# Now drop the SiteAddress column
Beverly_Grove_df.drop(['SITEADDRESS'], axis=1, inplace=True)

In [559]:
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,0,0,0,0,0,0,0,0,0,0


In [560]:
# Step 1: Create our X and y

y = Beverly_Grove_df['Did it sell?']
X = Beverly_Grove_df.drop(['Did it sell?'], 1)

In [561]:
APN = Beverly_Grove_df.index.values
APN

array([5085010022, 5085033008, 5085032022, ..., 5510018012, 5510017023,
       5514001029])

In [562]:
# Step 2: Use PCA to reduce dimension to three principal components.
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

In [563]:
X_pca

array([[ 1037442.40088321,   222949.06197813,    99008.44211243],
       [ -932464.7141857 ,   251848.62995472,   -90439.60157539],
       [ -393042.03210882,  -290764.40544306,   -27460.470998  ],
       ...,
       [ -314557.49024403,  -369458.72763661,   -27020.9155848 ],
       [ 2038277.22062523,   419857.55993186, -1288530.60059949],
       [ -962055.78924987,   281605.81570128,   -93574.2647082 ]])

In [564]:
# Step 3: Train, test, split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, random_state=78, stratify=y)

In [565]:
# Step 4: Scale our data

# Create the StandardScaler instance
scaler = StandardScaler()

# Fit our scaler, named 'scaler' to our data, which produces a new StandardScaler object
# which we call 'X_scaler'
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [566]:
X_train_scaled_APN = []

In [567]:
# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_random_oversampled, y_random_oversampled = ros.fit_resample(X_train_scaled, y_train)

Counter(y_random_oversampled)

Counter({0: 3287, 1: 3287})

In [568]:
# Logistic regression using random oversampled data
from sklearn.linear_model import LogisticRegression

model_10 = LogisticRegression(solver='lbfgs', 
                                max_iter=300,
                                random_state=78,
                                class_weight="balanced")

model_10.fit(X_random_oversampled, y_random_oversampled)

LogisticRegression(class_weight='balanced', max_iter=300, random_state=78)

In [569]:
# Evaluate the model
y_pred = model_10.predict(X_test_scaled)

In [570]:
print(f" Logistic regression model accuracy: {accuracy_score(y_test, y_pred):.3f}")

 Logistic regression model accuracy: 0.621


In [571]:
# just for shits n' gigs, lets find the balanced accuracy score

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.49355118500495626

In [572]:
f1_score(y_test, y_pred, average="weighted")

0.7146103073370367

In [573]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[703, 393],
       [ 53,  28]])

In [224]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.93      0.64      0.35      0.76      0.47      0.23      1096
          1       0.07      0.35      0.64      0.11      0.47      0.22        81

avg / total       0.87      0.62      0.37      0.71      0.47      0.23      1177



In [225]:
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test, "PARCEL": y_test.index.values}).reset_index(drop=True)
results.sample(20)

,Prediction,Actual,PARCEL
786,1,0,5510020027
54,0,1,5527034020
659,0,0,4305003025
799,0,0,4335003011
346,0,0,5514009035
60,0,1,5510014003
955,0,0,5089021001
15,1,0,5085003005
761,0,0,5510007001
780,0,0,5512021001


In [226]:
Beverly_Grove_Results_SFRs_df = Beverly_Grove_df.merge(results, how="left", right_on="PARCEL", left_on="PARCEL")
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,1,0,0,85697,74539,11158,0,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,0,0,0,62487,42013,20474,0,1,...,0,0,0,0,0,0,0,0,0,0


In [228]:
Beverly_Grove_Results_SFRs_df = pd.DataFrame(Beverly_Grove_Results_SFRs_df, columns = ['PARCEL', 'Did it sell?', 'Prediction', 'Actual'])
Beverly_Grove_Results_SFRs_df.head()

,PARCEL,Did it sell?,Prediction,Actual
0,5085010022,0,NaN,NaN
1,5085033008,0,0.0,0.0
2,5085032022,0,NaN,NaN
3,5510005006,0,0.0,0.0
4,5089024002,0,NaN,NaN


In [229]:
Beverly_Grove_Results_SFRs_df.to_csv('Beverly_Grove_Results_SFRs.csv')

# Now do the Beverly Grove Condos

In [574]:
# bring in our dataframe

Beverly_Grove_df = pd.read_csv(path, index_col='PARCEL')
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,6727 Drexel Ave,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,942 S Cochran Ave,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,0,0


In [575]:
# drop any nulls

Beverly_Grove_df = Beverly_Grove_df.dropna()

In [576]:
# Change the datatype of these two columns into ints

Beverly_Grove_df['TAXAMT'] = Beverly_Grove_df['TAXAMT'].astype(int)
Beverly_Grove_df['EFFYRBLT'] = Beverly_Grove_df['EFFYRBLT'].astype(int)

In [577]:
# Make sure our dataframe is only Condos so exclude every property that doesn't have a '#' in its site address

Beverly_Grove_df = Beverly_Grove_df[Beverly_Grove_df['SITEADDRESS'].str.contains('#')]
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5087012023,0,0,1040 S La Jolla Ave #VE,0,0,0,289000,116000,173000,0,...,0,0,0,0,0,0,0,1,0,0
4332020041,0,0,1045 S Bedford St #9,1,0,0,151323,41266,110057,0,...,0,0,0,0,0,0,0,0,0,0
4332020051,0,0,1045 S Bedford St #1,0,0,0,376863,223789,153074,0,...,0,0,0,0,0,0,0,0,0,0


In [578]:
# Now drop the SiteAddress column
Beverly_Grove_df.drop(['SITEADDRESS'], axis=1, inplace=True)

In [579]:
# Step 1: Create our X and y

y = Beverly_Grove_df['Did it sell?']
X = Beverly_Grove_df.drop(['Did it sell?'], 1)

In [580]:
APN = Beverly_Grove_df.index.values
APN

array([5087012023, 4332020041, 4332020051, ..., 5089003038, 5089003039,
       5089003040])

In [581]:
# Step 2: Use PCA to reduce dimension to three principal components.
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

In [582]:
# Step 3: Train, test, split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, random_state=78, stratify=y)

In [583]:
# Step 4: Scale our data

# Create the StandardScaler instance
scaler = StandardScaler()

# Fit our scaler, named 'scaler' to our data, which produces a new StandardScaler object
# which we call 'X_scaler'
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [584]:
X_train_scaled_APN = []

In [585]:
# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_random_oversampled, y_random_oversampled = ros.fit_resample(X_train_scaled, y_train)

Counter(y_random_oversampled)

Counter({0: 1380, 1: 1380})

In [586]:
# Logistic regression using random oversampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', 
                                max_iter=300,
                                random_state=78,
                                class_weight="balanced")

model.fit(X_random_oversampled, y_random_oversampled)

LogisticRegression(class_weight='balanced', max_iter=300, random_state=78)

In [587]:
# Evaluate the model
y_pred = model.predict(X_test_scaled)

In [588]:
print(f" Logistic regression model accuracy: {accuracy_score(y_test, y_pred):.3f}")

 Logistic regression model accuracy: 0.566


In [589]:
# just for shits n' gigs, lets find the balanced accuracy score

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.48128019323671495

In [590]:
f1_score(y_test, y_pred, average="weighted")

0.6593455279133104

In [248]:
# See our results in a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[269, 191],
       [ 28,  17]])

In [249]:
# evaluate
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.58      0.71       460
           1       0.08      0.38      0.13        45

    accuracy                           0.57       505
   macro avg       0.49      0.48      0.42       505
weighted avg       0.83      0.57      0.66       505



In [250]:
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test, "PARCEL": y_test.index.values}).reset_index(drop=True)
results.sample(20)

,Prediction,Actual,PARCEL
25,0,0,5510027049
489,1,0,4333014056
375,1,0,4335008149
353,1,0,4334009045
503,0,0,4332021057
435,0,0,5089011092
420,0,1,4335012133
143,1,0,4335001053
356,0,0,4333014043
454,0,0,4335004032


In [251]:
Beverly_Grove_Results_Condos_df = Beverly_Grove_df.merge(results, how="left", right_on="PARCEL", left_on="PARCEL")
Beverly_Grove_Results_Condos_df.head()

,PARCEL,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY,Prediction,Actual
0,5087012023,0,0,0,0,0,289000,116000,173000,0,...,0,0,0,0,0,1,0,0,0.0,0.0
1,4332020041,0,0,1,0,0,151323,41266,110057,0,...,0,0,0,0,0,0,0,0,0.0,0.0
2,4332020051,0,0,0,0,0,376863,223789,153074,0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,4332020042,0,0,1,0,1,255813,148748,107065,0,...,0,0,0,0,0,0,0,0,NaN,NaN
4,4332020043,1,0,1,0,1,176090,38515,137575,0,...,0,0,1,0,0,0,0,0,NaN,NaN


In [252]:
Beverly_Grove_Results_Condos_df = pd.DataFrame(Beverly_Grove_Results_Condos_df, columns = ['PARCEL', 'Did it sell?', 'Prediction', 'Actual'])
Beverly_Grove_Results_Condos_df.head(3)

,PARCEL,Did it sell?,Prediction,Actual
0,5087012023,0,0.0,0.0
1,4332020041,0,0.0,0.0
2,4332020051,0,NaN,NaN


In [254]:
Beverly_Grove_Results_Condos_df.to_csv('Beverly_Grove_Results_Condos.csv')

# Attempt 17: Same as Attempt 16, but Oversample before splitting

In [518]:
# bring in our dataframe

Beverly_Grove_df = pd.read_csv(path, index_col='PARCEL')
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,6727 Drexel Ave,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,942 S Cochran Ave,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,0,0


In [519]:
# drop any nulls

Beverly_Grove_df = Beverly_Grove_df.dropna()

In [520]:
# Change the datatype of these two columns into ints

Beverly_Grove_df['TAXAMT'] = Beverly_Grove_df['TAXAMT'].astype(int)
Beverly_Grove_df['EFFYRBLT'] = Beverly_Grove_df['EFFYRBLT'].astype(int)

In [521]:
# Make sure our dataframe is only SFR's so exclude every property that has an '#' in its site address

Beverly_Grove_df = Beverly_Grove_df[~Beverly_Grove_df['SITEADDRESS'].str.contains('#')]
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0


In [522]:
# Now drop the SiteAddress column
Beverly_Grove_df.drop(['SITEADDRESS'], axis=1, inplace=True)

In [523]:
Beverly_Grove_df.head(3)

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,0,0,0,0,0,0,0,0,0,0


In [537]:
# Step 1: Create our X and y

y_17 = Beverly_Grove_df['Did it sell?']
X_17 = Beverly_Grove_df.drop(['Did it sell?'], 1)

In [538]:
APN = Beverly_Grove_df.index.values
APN

array([5085010022, 5085033008, 5085032022, ..., 5510018012, 5510017023,
       5514001029])

In [539]:
# Step 2: Resample our data via a Random Oversampler of our minority class

# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_random_oversampled, y_random_oversampled = ros.fit_resample(X_17, y_17)

Counter(y_random_oversampled)

Counter({0: 4383, 1: 4383})

In [540]:
len(y_random_oversampled)

8766

In [541]:
# Step 3: Do our PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_random_oversampled)

In [542]:
len(X_pca)

8766

In [543]:
len(y_random_oversampled)

8766

In [545]:
# Step 4: Train, test, split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y_random_oversampled, random_state=78)

In [546]:
# Step 5: Scale our data

# Create the StandardScaler instance
scaler = StandardScaler()

# Fit our scaler, named 'scaler' to our data, which produces a new StandardScaler object
# which we call 'X_scaler'
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [547]:
X_train_scaled_APN = []

In [548]:
# Logistic regression using random oversampled data
from sklearn.linear_model import LogisticRegression

model_11 = LogisticRegression(solver='lbfgs', 
                                max_iter=300,
                                random_state=78,
                                class_weight="balanced")

model_11.fit(X_train_scaled, y_train)

LogisticRegression(class_weight='balanced', max_iter=300, random_state=78)

In [549]:
# Evaluate the model
y_pred = model_11.predict(X_test_scaled)

In [550]:
print(f" Logistic regression model accuracy: {accuracy_score(y_test, y_pred):.3f}")

 Logistic regression model accuracy: 0.523


In [551]:
# just for shits n' gigs, lets find the balanced accuracy score

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.521507498026835

In [552]:
f1_score(y_test, y_pred, average="weighted")

0.504802731589642

In [553]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.52      0.71      0.33      0.60      0.48      0.24      1106
          1       0.53      0.33      0.71      0.41      0.48      0.23      1086

avg / total       0.53      0.52      0.52      0.50      0.48      0.23      2192

